In [1]:
import os
import random
from PIL import Image
from tqdm import tqdm

In [2]:
def place_chips_on_background(chips_paths, background_path, output_path):
    # Load the background image
    background = Image.open(background_path).convert("RGBA")

    # Background dimensions
    bg_width, bg_height = background.size

    for chip_path in chips_paths:
        # Load and process each chip
        chip = Image.open(chip_path).convert("RGBA")
        
        # Randomly rotate the chip
        angle = random.randint(0, 359)
        chip = chip.rotate(angle, expand=1)

        # Determine a random scale factor based on the background width
        scale_factor = random.uniform(0.1, 0.3)
        new_chip_width = int(bg_width * scale_factor)

        # Calculate new height to maintain aspect ratio
        chip_aspect_ratio = chip.width / chip.height
        new_chip_height = int(new_chip_width / chip_aspect_ratio)

        # Resize the chip
        chip = chip.resize((new_chip_width, new_chip_height), Image.Resampling.LANCZOS)

        # Randomly position the resized chip on the background
        max_x, max_y = bg_width - new_chip_width, bg_height - new_chip_height
        pos_x = random.randint(0, max_x)
        pos_y = random.randint(0, max_y)

        # Extract the alpha channel from the chip as the transparency mask
        mask = chip.split()[3]

        # Paste the resized chip onto the background using the mask
        background.paste(chip, (pos_x, pos_y), mask)

    # Save the combined image to the output path
    background.save(output_path)

In [3]:
def make_dataset(background_dir, chips_dir, output_dir, num_samples):
    # Lists to hold the file paths for chips and backgrounds
    chips = []
    backgrounds = []

    # Go through the backgrounds directory to collect background images
    for background_file in os.listdir(background_dir):
        backgrounds.append(os.path.join(background_dir, background_file))

    # Go through all the subdirectories in the chips directory to collect chip images
    for root, dirs, files in tqdm(os.walk(chips_dir), desc="Loading Chips"):
        for chip_file in files:
            chips.append(os.path.join(root, chip_file))

    for i in tqdm(range(num_samples), desc="Creating Dataset"):
        # Randomly select a number of chips and a background
        num_chips = random.randint(2, 10)
        selected_chips = random.sample(chips, num_chips)
        selected_background = random.choice(backgrounds)

        output_image_path = os.path.join(output_dir, f"{i}.png")

        # Place the chips on the background
        place_chips_on_background(selected_chips, selected_background, output_image_path)

In [4]:
background_dir = "../Backgrounds"
chips_dir = "../Chips/CroppedChips"
output_dir = "../Dataset/AllChips/Multiples"
num_samples = 100


make_dataset(background_dir, chips_dir, output_dir, num_samples)

Loading Chips: 1it [00:00, ?it/s]
Creating Dataset: 100%|██████████████████████████████████████████████████████████████| 100/100 [00:06<00:00, 15.03it/s]
